### Caracterização e visualização de dados

In [9]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures,OneHotEncoder
from matplotlib import pyplot as plt
from category_encoders.one_hot import OrdinalEncoder
import category_encoders as ce
from sklearn.model_selection import train_test_split

In [51]:
newDataframe = pd.read_csv("test_categorical_variables.csv")

#y = newDataframe['SalePrice']
y = pd.DataFrame(newDataframe['SalePrice'])
X = newDataframe.drop(['SalePrice'], axis=1)

y[["SalePrice"]] = y[["SalePrice"]].apply(np.log)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [82]:
from sklearn.model_selection import cross_validate, KFold

def validacao_cruzada2(modelo):
  
  kf = KFold(n_splits=10, shuffle=True, random_state=3)
  scores = cross_validate(modelo, X_train, y_train, cv=kf, scoring =('r2', 'neg_mean_squared_error'), return_train_score=True)
  return scores

In [83]:
def f_rmse(y_real, y_pred): 
    return mean_squared_error(y_real, y_pred)**0.5

def avalia_classificador(clf, kf, X, y, f_metrica):
    metrica_valid = []
    metrica_train = []
    r2_valid = []
    r2_train = []
    
    y_preds = np.zeros(X.shape[0])
    
    for train, valid in kf.split(X,y):
        x_train = X.iloc[train] 
        y_train = y.iloc[train]
        x_valid = X.iloc[valid] 
        y_valid = y.iloc[valid]
        clf.fit(x_train, y_train) 
        y_pred_train = clf.predict(x_train)
        y_pred_valid = clf.predict(x_valid)
        y_preds[valid] = y_pred_valid[0] 
        
        metrica_valid.append(f_metrica(y_valid, y_pred_valid)) 
        metrica_train.append(f_metrica(y_train, y_pred_train))
        r2_valid.append(r2_score(y_valid, y_pred_valid))
        r2_train.append(r2_score(y_train, y_pred_train))
          
    # retorna as previsões e a média das métricas de treino e validação
    # obtidas nas iterações do Kfold
    return y_preds, np.array(metrica_valid).mean(), np.array(metrica_train).mean(), np.array(r2_valid).mean(), np.array(r2_train).mean()

In [84]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import KFold

regressor_linear = LinearRegression(fit_intercept = True)

regressor_linear = regressor_linear.fit(X_train, y_train)

y_resposta_treino = regressor_linear.predict(X_train)
y_resposta_teste  = regressor_linear.predict(X_test)

print(' ')
print(' REGRESSOR LINEAR:')
print(' ')

print(' Métrica  DENTRO da amostra  FORA da amostra')
print(' -------  -----------------  ---------------')

mse_in  = mean_squared_error(y_train,y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_train,y_resposta_treino)

mse_out  = mean_squared_error(y_test,y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_test,y_resposta_teste)

print(' %7s  %17.4f  %15.4f' % (  'mse' ,  mse_in ,  mse_out ) )
print(' %7s  %17.4f  %15.4f' % ( 'rmse' , rmse_in , rmse_out ) )
print(' %7s  %17.4f  %15.4f' % (   'r2' ,   r2_in ,   r2_out ) )

kf = KFold(n_splits=9, shuffle=True, random_state=7)

preds, rmse_val, rmse_train, r2test, r2train = avalia_classificador(regressor_linear, kf, X, y, f_rmse)

print(' ')
print(' VALIDAÇÃO CRUZADA:')
print(' ')

print(' Métrica  DENTRO da amostra  FORA da amostra')
print(' -------  -----------------  ---------------')

print(' %7s  %17.4f  %15.4f' % ( 'rmse' , rmse_train , rmse_val ) )
print(' %7s  %17.4f  %15.4f' % (   'r2' ,   r2train ,   r2test ) )

sco = validacao_cruzada2(regressor_linear)
print(sorted(sco.keys()))

import statistics
mean = statistics.mean(sco['test_r2'])
print(sco['test_r2'])
print(mean)
print(sco['train_r2'])
print("RMSE:", np.sqrt(mean_squared_error(y_test, pred_test_tree)))
#print(sco['train_score'])


 
 REGRESSOR LINEAR:
 
 Métrica  DENTRO da amostra  FORA da amostra
 -------  -----------------  ---------------
     mse             0.0001           0.0001
    rmse             0.0082           0.0109
      r2             0.9376           0.8980
 Validação cruzada
 Métrica  DENTRO da amostra  FORA da amostra
 -------  -----------------  ---------------
    rmse             0.0083           0.0123
      r2             0.9371           0.8500
['fit_time', 'score_time', 'test_neg_mean_squared_error', 'test_r2', 'train_neg_mean_squared_error', 'train_r2']
[0.89503472 0.89884523 0.41682026 0.73852389 0.88673919 0.93061628
 0.73386645 0.86199354 0.89352611 0.91408331]
0.817004898121417
[0.93883629 0.93764915 0.94053019 0.94541864 0.93873627 0.93506722
 0.94608051 0.94164028 0.93992036 0.93673868]


In [85]:
# Distribuições, correlações, etc.

### Validação cruzada

In [86]:
# Descrição do procedimento de validação

### Regressão Linear

### Regressão Logística

In [20]:
#Não se aplica

### Regressão Bayesiana

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.linear_model import BayesianRidge, LinearRegression


bayesian_ridge = BayesianRidge(compute_score=True)

bayesian_ridge = bayesian_ridge.fit(X_train, y_train.values.ravel())

y_resposta_treino = bayesian_ridge.predict(X_train)
y_resposta_teste  = bayesian_ridge.predict(X_test)

print(' ')
print(' REGRESSÃO BAYESIANA:')
print(' ')

print(' Métrica  DENTRO da amostra  FORA da amostra')
print(' -------  -----------------  ---------------')

mse_in  = mean_squared_error(y_train,y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_train,y_resposta_treino)

mse_out  = mean_squared_error(y_test,y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_test,y_resposta_teste)

print(' %7s  %17.4f  %15.4f' % (  'mse' ,  mse_in ,  mse_out ) )
print(' %7s  %17.4f  %15.4f' % ( 'rmse' , rmse_in , rmse_out ) )
print(' %7s  %17.4f  %15.4f' % (   'r2' ,   r2_in ,   r2_out ) )

 
 REGRESSÃO BAYESIANA:
 
 Métrica  DENTRO da amostra  FORA da amostra
 -------  -----------------  ---------------
     mse             0.0001           0.0001
    rmse             0.0099           0.0111
      r2             0.9080           0.8954


### Árvores de decisão

In [22]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
#from sklearn.metrics import mean_squared_error, r2_score 
from sklearn import metrics
#import math


# Create Decision Tree classifer object
clf = DecisionTreeRegressor()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_resposta_treino = clf.predict(X_train)
y_resposta_teste = clf.predict(X_test)

print(' ')
print(' ARVORE DE DECISÃO:')
print(' ')

print(' Métrica  DENTRO da amostra  FORA da amostra')
print(' -------  -----------------  ---------------')

mse_in  = mean_squared_error(y_train,y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_train,y_resposta_treino)

mse_out  = mean_squared_error(y_test,y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_test,y_resposta_teste)

print(' %7s  %17.4f  %15.4f' % (  'mse' ,  mse_in ,  mse_out ) )
print(' %7s  %17.4f  %15.4f' % ( 'rmse' , rmse_in , rmse_out ) )
print(' %7s  %17.4f  %15.4f' % (   'r2' ,   r2_in ,   r2_out ) )
#print("Acurácia:",metrics.accuracy_score(y_test, y_pred))

 
 ARVORE DE DECISÃO:
 
 Métrica  DENTRO da amostra  FORA da amostra
 -------  -----------------  ---------------
     mse             0.0000           0.0003
    rmse             0.0000           0.0174
      r2             1.0000           0.7423


### Random Forest

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
#import math

# Create Decision Tree classifer object
clf = RandomForestRegressor()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train.values.ravel())

#Predict the response for test dataset
y_resposta_treino = clf.predict(X_train)
y_resposta_teste = clf.predict(X_test)

print(' ')
print(' RANDOM FOREST:')
print(' ')

print(' Métrica  DENTRO da amostra  FORA da amostra')
print(' -------  -----------------  ---------------')

mse_in  = mean_squared_error(y_train,y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_train,y_resposta_treino)

mse_out  = mean_squared_error(y_test,y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_test,y_resposta_teste)

print(' %7s  %17.4f  %15.4f' % (  'mse' ,  mse_in ,  mse_out ) )
print(' %7s  %17.4f  %15.4f' % ( 'rmse' , rmse_in , rmse_out ) )
print(' %7s  %17.4f  %15.4f' % (   'r2' ,   r2_in ,   r2_out ) )
#print("Acurácia:",metrics.accuracy_score(y_test, y_pred))

 
 RANDOM FOREST:
 
 Métrica  DENTRO da amostra  FORA da amostra
 -------  -----------------  ---------------
     mse             0.0000           0.0001
    rmse             0.0048           0.0117
      r2             0.9789           0.8824


### Gradient Boosting

In [24]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
#import math

# Create Decision Tree classifer object
clf = GradientBoostingRegressor(random_state=0)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train.values.ravel())

#Predict the response for test dataset
y_resposta_treino = clf.predict(X_train)
y_resposta_teste = clf.predict(X_test)

print(' ')
print(' GRADIENT BOOSTING:')
print(' ')

print(' Métrica  DENTRO da amostra  FORA da amostra')
print(' -------  -----------------  ---------------')

mse_in  = mean_squared_error(y_train,y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_train,y_resposta_treino)

mse_out  = mean_squared_error(y_test,y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_test,y_resposta_teste)

print(' %7s  %17.4f  %15.4f' % (  'mse' ,  mse_in ,  mse_out ) )
print(' %7s  %17.4f  %15.4f' % ( 'rmse' , rmse_in , rmse_out ) )
print(' %7s  %17.4f  %15.4f' % (   'r2' ,   r2_in ,   r2_out ) )
#print("Acurácia:",metrics.accuracy_score(y_test, y_pred))
"""
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from numpy import mean,std

model = GradientBoostingClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Acurácia: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
"""

 
 GRADIENT BOOSTING:
 
 Métrica  DENTRO da amostra  FORA da amostra
 -------  -----------------  ---------------
     mse             0.0000           0.0001
    rmse             0.0061           0.0107
      r2             0.9652           0.9020


"\nfrom sklearn.ensemble import GradientBoostingClassifier\nfrom sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score\nfrom numpy import mean,std\n\nmodel = GradientBoostingClassifier()\ncv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)\nn_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')\nprint('Acurácia: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))\n"

### Support Vector Machines

In [25]:
from sklearn.svm import SVR
from sklearn import metrics
#import math

# Fit regression model
svr_rbf = SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1)
svr_lin = SVR(kernel="linear", C=100, gamma="auto")
svr_poly = SVR(kernel="poly", C=100, gamma="auto", degree=3, epsilon=0.1, coef0=1)

# Create Decision Tree classifer object
#clf = GradientBoostingRegressor(random_state=0)

# Train Decision Tree Classifer
svr_rbf = svr_rbf.fit(X_train,y_train.values.ravel())

#Predict the response for test dataset
y_resposta_treino = svr_rbf.predict(X_train)
y_resposta_teste = svr_rbf.predict(X_test)

print(' ')
print(' SUPPORT VECTOR MACHINES RBF KERNEL:')
print(' ')

print(' Métrica  DENTRO da amostra  FORA da amostra')
print(' -------  -----------------  ---------------')

mse_in  = mean_squared_error(y_train,y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_train,y_resposta_treino)

mse_out  = mean_squared_error(y_test,y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_test,y_resposta_teste)

print(' %7s  %17.4f  %15.4f' % (  'mse' ,  mse_in ,  mse_out ) )
print(' %7s  %17.4f  %15.4f' % ( 'rmse' , rmse_in , rmse_out ) )
print(' %7s  %17.4f  %15.4f' % (   'r2' ,   r2_in ,   r2_out ) )

#******************************************************************

# Train Decision Tree Classifer
svr_lin = svr_lin.fit(X_train,y_train.values.ravel())

#Predict the response for test dataset
y_resposta_treino = svr_lin.predict(X_train)
y_resposta_teste = svr_lin.predict(X_test)

print(' ')
print(' SUPPORT VECTOR MACHINES LINEAR KERNEL:')
print(' ')

print(' Métrica  DENTRO da amostra  FORA da amostra')
print(' -------  -----------------  ---------------')

mse_in  = mean_squared_error(y_train,y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_train,y_resposta_treino)

mse_out  = mean_squared_error(y_test,y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_test,y_resposta_teste)

print(' %7s  %17.4f  %15.4f' % (  'mse' ,  mse_in ,  mse_out ) )
print(' %7s  %17.4f  %15.4f' % ( 'rmse' , rmse_in , rmse_out ) )
print(' %7s  %17.4f  %15.4f' % (   'r2' ,   r2_in ,   r2_out ) )

#***************************************************************

# Train Decision Tree Classifer
svr_poly = svr_poly.fit(X_train,y_train.values.ravel())

#Predict the response for test dataset
y_resposta_treino = svr_poly.predict(X_train)
y_resposta_teste = svr_poly.predict(X_test)

print(' ')
print(' SUPPORT VECTOR MACHINES POLYNOMIAL KERNEL:')
print(' ')

print(' Métrica  DENTRO da amostra  FORA da amostra')
print(' -------  -----------------  ---------------')

mse_in  = mean_squared_error(y_train,y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_train,y_resposta_treino)

mse_out  = mean_squared_error(y_test,y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_test,y_resposta_teste)

print(' %7s  %17.4f  %15.4f' % (  'mse' ,  mse_in ,  mse_out ) )
print(' %7s  %17.4f  %15.4f' % ( 'rmse' , rmse_in , rmse_out ) )
print(' %7s  %17.4f  %15.4f' % (   'r2' ,   r2_in ,   r2_out ) )


 
 SUPPORT VECTOR MACHINES RBF KERNEL:
 
 Métrica  DENTRO da amostra  FORA da amostra
 -------  -----------------  ---------------
     mse             0.0012           0.0013
    rmse             0.0342           0.0356
      r2            -0.0889          -0.0804
 
 SUPPORT VECTOR MACHINES LINEAR KERNEL:
 
 Métrica  DENTRO da amostra  FORA da amostra
 -------  -----------------  ---------------
     mse             0.0011           0.0012
    rmse             0.0335           0.0340
      r2            -0.0455           0.0132
 
 SUPPORT VECTOR MACHINES POLYNOMIAL KERNEL:
 
 Métrica  DENTRO da amostra  FORA da amostra
 -------  -----------------  ---------------
     mse             0.0014           0.0014
    rmse             0.0378           0.0380
      r2            -0.3344          -0.2315


### Redes Neurais

In [38]:


from sklearn.model_selection import cross_validate, KFold

def validacao_cruzada(modelo):
  
  kf = KFold(n_splits=10)
  scores = cross_validate(modelo, X_train, y_train, cv=kf, scoring='r2', return_train_score=True)
  return scores


from sklearn.neural_network import MLPRegressor
#regr = MLPRegressor(random_state=64, max_iter=50000).fit(X_train, y_train.values.ravel())

regr = MLPRegressor(random_state=1234, solver='adam', activation='relu')

score1 = validacao_cruzada(regr)
#Predict the response for test dataset
y_resposta_treino = regr.predict(X_train)
y_resposta_teste = regr.predict(X_test)

print(' ')
print(' NEURAL NETWORK:')
print(' ')

print(' Métrica  DENTRO da amostra  FORA da amostra')
print(' -------  -----------------  ---------------')

mse_in  = mean_squared_error(y_train,y_resposta_treino)
rmse_in = math.sqrt(mse_in)
r2_in   = r2_score(y_train,y_resposta_treino)

mse_out  = mean_squared_error(y_test,y_resposta_teste)
rmse_out = math.sqrt(mse_out)
r2_out   = r2_score(y_test,y_resposta_teste)

print(' %7s  %17.4f  %15.4f' % (  'mse' ,  mse_in ,  mse_out ) )
print(' %7s  %17.4f  %15.4f' % ( 'rmse' , rmse_in , rmse_out ) )
print(' %7s  %17.4f  %15.4f' % (   'r2' ,   r2_in ,   r2_out ) )

/opt/anaconda/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda/lib/pyth

NotFittedError: This MLPRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [26]:
# Redes Neurais